In [12]:

import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
X_data, y_data = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False, parser="pandas")
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)


In [23]:
from torch.utils.data import Dataset, DataLoader

import torch 
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

In [1]:
torch.cuda.get_device_name(0)

NameError: name 'torch' is not defined

In [18]:
class TrainDataset(Dataset):
    def __init__(self, X, y):
        self.X = X.reshape(-1, 1, 28, 28)
        self.y = y.astype(int)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        
        # Normalize the X data
        self.X = self.X / 255.0

        return self.X[idx], self.y[idx]
    
class TestDataset(Dataset):
    def __init__(self, X):
        self.X = X.reshape(-1, 1, 28, 28)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        
        # Normalize the X data
        self.X = self.X / 255.0

        return self.X[idx]

In [21]:
train_set = TrainDataset(X_train, y_train)

test_set = TestDataset(X_test)

In [22]:
batch_size = 512

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [ ]:
class MultiLayerPerceptron(nn.Module):
    def __init__(self):
        super(MultiLayerPerceptron, self).__init__()
        
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits